In [2]:
%cd '/kaggle/input/dl-th-da/proj/'

# Prepare data

In [3]:
from torchvision import datasets
import torch.backends.cudnn as cudnn
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.optim as optim

import pandas as pd
import numpy as np
import os
from skimage import io
#from torchvision.io import read_image

import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [4]:
root = 'dataset'
IMG_SIZE = 32
BATCH_SIZE = 32
lr = 1e-3
EPOCHS = 200

In [132]:
#source_images = np.load('dataset/X_new_rgb.npy')
#source_labels = np.load('dataset/Y_c1.npy')

source_images = np.load('dataset/X_aug.npy')
source_labels = np.load('dataset/Y_aug_c.npy')

target_images = np.load('dataset/X_new_th.npy')
target_labels = np.load('dataset/Y.npy')

source_images.shape, target_images.shape, source_labels.shape, target_labels.shape

In [6]:
source_images.dtype, source_labels.dtype

In [133]:
target_labels = np.squeeze(target_labels,1)
target_images.shape, target_labels.shape

In [134]:
from sklearn.model_selection import train_test_split

source_train_images, source_test_images, source_train_labels, source_test_labels = train_test_split(source_images, source_labels, test_size=0.2, random_state=42, stratify=source_labels)
target_train_images, target_test_images, target_train_labels, target_test_labels = train_test_split(target_images, target_labels, test_size=0.6, random_state=42, stratify=target_labels)

In [135]:
len(source_train_images), len(target_train_images), len(source_test_images), len(target_test_images), source_train_images.dtype, source_train_labels.dtype

In [10]:
source_train_images.shape

In [11]:
from PIL import Image
class CustomDataset(Dataset):

    def __init__(self, img_array, label_array, transform=None):

        self.labels = label_array#np.load(label_file)
        self.images = img_array#np.load(img_file)
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = self.images[idx]#.astype(int)
        PIL_image = Image.fromarray(image.astype('uint8'), 'RGB')
        label = self.labels[idx]#.astype(int)
        #sample = {'image': image, 'label': label}

        if self.transform:
            image = self.transform(PIL_image)
            #image = image.transpose(2, 0, 1)
            #print('------------',image.shape)

        return image,label

In [12]:
img_size = 224
BATCH_SIZE = 32
preprocess = transforms.Compose([
    transforms.Resize((img_size,img_size)),
    #transforms.CenterCrop(299),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.675 , 0.646, 0.616], std=[0.141, 0.158 , 0.186]),
    ])

preprocess_target = transforms.Compose([
  transforms.Resize((img_size,img_size)),
  #transforms.CenterCrop(299),
  transforms.ToTensor(),
  #transforms.Normalize(mean=[0.413, 0.413, 0.413], std=[0.215, 0.215, 0.215]),
])

In [136]:
train = CustomDataset(
    img_array=source_train_images,
    label_array=source_train_labels,
    transform=preprocess)

#torch.utils.data.TensorDataset(torch.from_numpy(img_rgb_train), torch.from_numpy(labels_rgb_train))
test = CustomDataset(
    img_array=source_test_images,
    label_array=source_test_labels,
    transform=preprocess)

train_th = CustomDataset(
    img_array=target_train_images,
    label_array=target_train_labels,
    transform=preprocess_target)
test_th = CustomDataset(
    img_array=target_test_images,
    label_array=target_test_labels,
    transform=preprocess_target)


In [137]:
dataloader_source_test = DataLoader(test, batch_size=BATCH_SIZE, shuffle=True)
dataloader_source = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
dataloader_target_test = DataLoader(test_th, batch_size=BATCH_SIZE, shuffle=True)
dataloader_target = DataLoader(train_th, batch_size=BATCH_SIZE, shuffle=True)

In [74]:
q = np.concatenate((qtrain_source,qtest_source), axis=0)
q.shape

In [75]:
len(dataloader_source)

In [80]:
for i in range(len(dataloader_source)):
    if(i==0):
        
        temp = next(iter(dataloader_source))
        qtrain_source = temp[0].numpy()
        qtrain_source_l = temp[1].numpy()
    else:
        temp = next(iter(dataloader_source))
        qtrain_source = np.concatenate((qtrain_source,temp[0].numpy()), axis=0)
        qtrain_source_l = np.concatenate((qtrain_source_l,temp[1].numpy()), axis=0)

In [82]:
for i in range(len(dataloader_source_test)):
    if(i==0):
        
        temp = next(iter(dataloader_source_test))
        qtest_source = temp[0].numpy()
        qtest_source_l = temp[1].numpy()
    else:
        temp = next(iter(dataloader_source_test))
        qtest_source = np.concatenate((qtest_source,temp[0].numpy()), axis=0)
        qtest_source_l = np.concatenate((qtest_source_l,temp[1].numpy()), axis=0)

In [84]:
for i in range(len(dataloader_target)):
    if(i==0):
        
        temp = next(iter(dataloader_target))
        qtrain_target = temp[0].numpy()
        qtrain_target_l = temp[1].numpy()
    else:
        temp = next(iter(dataloader_target))
        qtrain_target = np.concatenate((qtrain_target,temp[0].numpy()), axis=0)
        qtrain_target_l = np.concatenate((qtrain_target_l,temp[1].numpy()), axis=0)

In [86]:
for i in range(len(dataloader_target_test)):
    if(i==0):
        
        temp = next(iter(dataloader_target_test))
        qtest_target = temp[0].numpy()
        qtest_target_l = temp[1].numpy()
    else:
        temp = next(iter(dataloader_target_test))
        qtest_target = np.concatenate((qtest_target,temp[0].numpy()), axis=0)
        qtest_target_l = np.concatenate((qtest_target_l,temp[1].numpy()), axis=0)

In [87]:
qtrain_source.shape,qtrain_source_l.shape, qtest_source.shape,qtest_source_l.shape, qtrain_target.shape,qtrain_target_l.shape, qtest_target.shape,qtest_target_l.shape

In [94]:
!ls ../../../working

In [95]:
!mkdir ../../../working/datasets_244/

In [96]:
np.save('../../../working/datasets_244/atrain_source.npy', qtrain_source)
np.save('../../../working/datasets_244/atrain_source_labels.npy', qtrain_source_l)
np.save('../../../working/datasets_244/atest_source.npy', qtest_source)
np.save('../../../working/datasets_244/atest_source_labels.npy', qtest_source_l)
np.save('../../../working/datasets_244/atrain_target.npy', qtrain_target)
np.save('../../../working/datasets_244/atrain_target_labels.npy', qtrain_target_l)
np.save('../../../working/datasets_244/atest_target.npy', qtest_target)
np.save('../../../working/datasets_244/atest_target_labels.npy', qtest_target_l)

In [131]:
np.save('../../../working/datasets_244/test_target_labels.npy', qtest_target_l)
np.save('../../../working/datasets_244/test_target.npy', qtest_target)

In [138]:
len(dataloader_source), len(dataloader_source_test), len(dataloader_target), len(dataloader_target_test)

In [ ]:
52*32#1649*52
40*32#1280, 
13*32#413
60*32#,1920

In [ ]:
#dataset from numpy, resnet, densenet, alexnet, mobilenet, vgg for transfer learning

# Model

In [16]:
import torch
import math

#Ref: https://pytorch.org/tutorials/beginner/examples_autograd/polynomial_custom_function.html
class GRL(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, features,lambda_):
        ctx.lambda_ = lambda_  #ctx.save_for_backward(input)
        return features
    
    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.neg() * ctx.lambda_, None #input, = ctx.saved_tensors

In [139]:
from torchvision import models
pre =  models.alexnet(pretrained=True)
#models.mobilenet_v3_small(pretrained=True)
#models.resnet18(pretrained=True)
#models.alexnet(pretrained=True)

In [140]:
pre

In [141]:
i=0
for p in pre.features.parameters():
    print(i,p.shape)#requires_grad = False - Use this to determine number to freeze in model below
    i+=1

In [142]:
#Ref: https://towardsdatascience.com/practical-transfer-learning-with-pytorch-8344e5c82f59
class UDA(nn.Module):
    def __init__(self, base_model):
        super(UDA, self).__init__()
        
        self.feature = base_model.features 
        #self.avg_pool = nn.AdaptiveAvgPool2d(output_size=1)
        
        for i,p in enumerate(self.feature.parameters()):
            if(i<6): #[changed to [2,4,6] depending on how many conv layers to freeze], 132 for mobilenet
                p.requires_grad = False
        #current setting: all layers frozen
        '''resnet
        self.bm_conv1 = base_model.conv1
        self.bm_bn1 = base_model.bn1
        self.bm_relu = base_model.relu
        self.bm_maxpool = base_model.maxpool
        

        #self.feature =
        features = list(base_model.layer1)
        self.layer1 = nn.ModuleList(features)
        features = list(base_model.layer2)
        self.layer2 = nn.ModuleList(features)
        features = list(base_model.layer3)
        self.layer3 = nn.ModuleList(features)
        features = list(base_model.layer4)
        self.layer4 = nn.ModuleList(features)

        self.bm_avgpool = base_model.avgpool
        for i,p in enumerate(self.bm_conv1.parameters()):
            p.requires_grad = False
        for i,p in enumerate(self.bm_bn1.parameters()):
            p.requires_grad = False   
        for i,p in enumerate(self.layer1.parameters()):
            p.requires_grad = False
        for i,p in enumerate(self.layer2.parameters()):
            p.requires_grad = False
        for i,p in enumerate(self.layer3.parameters()):
            p.requires_grad = False
        for i,p in enumerate(self.layer4.parameters()):
            p.requires_grad = True
        #'''
        self.class_classifier = nn.Sequential()
        self.class_classifier.add_module('c_fc1', nn.Linear(256*6*6, 100))#256*6*6, 512, 576
        self.class_classifier.add_module('c_bn1', nn.BatchNorm1d(100))
        self.class_classifier.add_module('c_relu1', nn.ReLU(True))
        self.class_classifier.add_module('c_drop1', nn.Dropout2d())
        self.class_classifier.add_module('c_fc2', nn.Linear(100, 100))
        self.class_classifier.add_module('c_bn2', nn.BatchNorm1d(100))
        self.class_classifier.add_module('c_relu2', nn.ReLU(True))
        self.class_classifier.add_module('c_fc3', nn.Linear(100, 10))
        self.class_classifier.add_module('c_softmax', nn.LogSoftmax())

        self.domain_classifier = nn.Sequential()
        self.domain_classifier.add_module('d_fc1', nn.Linear(256*6*6, 100))#256*6*6
        self.domain_classifier.add_module('d_bn1', nn.BatchNorm1d(100))
        self.domain_classifier.add_module('d_relu1', nn.ReLU(True))
        self.domain_classifier.add_module('d_fc2', nn.Linear(100, 2))
        self.domain_classifier.add_module('d_softmax', nn.LogSoftmax(dim=1))
  
    # x represents our data
    def forward(self, x,lambda_):
        
        '''resnet
        feature = self.bm_conv1(x)
        feature = self.bm_bn1(feature)
        feature = self.bm_relu(feature)
        feature = self.bm_maxpool(feature)

        for layer_id, layer in enumerate(self.layer1):
            feature = layer(feature)
        for layer_id, layer in enumerate(self.layer2):
              feature = layer(feature)
        for layer_id, layer in enumerate(self.layer3):
              feature = layer(feature)
        for layer_id, layer in enumerate(self.layer4):
              feature = layer(feature)
        feature = self.bm_avgpool(feature)
        #'''

        feature = self.feature(x)
        #print(feature.shape)
        #feature = self.avg_pool(feature)
        #print(feature.shape)
        feature = feature.view(-1, np.prod(feature.shape[1:])) #256*6*6 for alexnet
        #print(feature.shape)
        grl = GRL.apply(feature, lambda_) #elementwise
        pred_label = self.class_classifier(feature)
        domain_label = self.domain_classifier(grl)
        return pred_label, domain_label

In [143]:
model = UDA(pre)

In [144]:
cuda = True
cudnn.benchmark = True

In [145]:
optimizer = torch.optim.Adam(model.parameters(),lr=lr)#, weight_decay=0.0005)

loss_label = torch.nn.NLLLoss()
loss_domain = torch.nn.NLLLoss()

if cuda:
    model = model.cuda()
    loss_label = loss_label.cuda()
    loss_domain = loss_domain.cuda()

In [23]:
def get_acc(pred, target):
    #preds =  pred.argmax(dim=1)
    preds = pred.data.max(1, keepdim=True)[1]
    correct = preds.eq(target.data.view_as(preds)).sum()
    
    return correct#/len(target)

# Train

In [146]:
def evaluate(model=model,dataloader_source_test=dataloader_source_test, dataloader_target_test=dataloader_target_test, BATCH_SIZE=BATCH_SIZE):
    len_dataloader = min(len(dataloader_source_test), len(dataloader_target_test))
    iter_source_test = iter(dataloader_source_test)
    iter_target_test = iter(dataloader_target_test)
    error_source_label=0
    error_source_domain=0
    error_target_label=0
    error_target_domain=0
    source_classification_acc = 0
    source_domain_acc = 0
    target_classification_acc = 0
    target_domain_acc = 0
    n_total=0
    
    
    for i in range(len_dataloader):
        
        p = float(i + len_dataloader)  / len_dataloader #ratio of training
        lambda_ = (2/(1 + np.exp(-10 * p))) -1
    
        img_source, lab_source  = iter_source_test.next()
        batch_size = len(lab_source)
        lab_domain = torch.zeros(batch_size).long()
        if cuda:
            img_source = img_source.cuda()
            lab_source = lab_source.cuda()
            lab_domain = lab_domain.cuda()
        pred_label, domain_label = model(img_source, lambda_)
        lab_source = lab_source.to(torch.long)
       
        error_source_label+= loss_label(pred_label, lab_source)
        error_source_domain += loss_domain(domain_label, lab_domain)
        
        #preds = pred_label.data.max(1, keepdim=True)[1]
        #correct_source_cls = preds.eq(lab_source.data.view_as(pred_label)).sum()
        n_total += BATCH_SIZE
    
        source_classification_acc+= get_acc(pred_label, lab_source)
        source_domain_acc += get_acc(domain_label, lab_domain)
    
        data_target = iter_target_test.next()
        img_target, lab_target = data_target
        batch_size = len(lab_target)
        lab_domain = torch.ones(batch_size).long()
        if cuda:
            img_target = img_target.cuda()
            lab_target = lab_target.cuda()
            lab_domain = lab_domain.cuda()
        pred_label, domain_label = model(img_target, lambda_)
        
        lab_target = lab_target.to(torch.long)
        error_target_label+= loss_label(pred_label, lab_target)
        error_target_domain += loss_domain(domain_label, lab_domain)
        
        target_classification_acc += get_acc(pred_label, lab_target)
        
        
        target_domain_acc += get_acc(domain_label, lab_domain)
    
    return (error_source_label/len_dataloader,error_source_domain/len_dataloader,error_target_label/len_dataloader,error_target_domain/len_dataloader,
           source_classification_acc/n_total, target_classification_acc/n_total,source_domain_acc/n_total, target_domain_acc/n_total )
       

In [154]:
from tqdm import tqdm
best_acc = 0
#src_err=[]
#src_d_err=[]
#tgt_d_err=[]
EPOCHS =200
#tgt_err=[]

#source_accu=[]
#target_accu=[]

for ep in tqdm(range(EPOCHS),desc='Epochs: '):
    
    len_dataloader = min(len(dataloader_source), len(dataloader_target))
    iter_source = iter(dataloader_source)
    iter_target = iter(dataloader_target)
    model.train(True)
    for i in range(len_dataloader):
        p = float(i + ep*len_dataloader) / EPOCHS / len_dataloader #ratio of training
        lambda_ = (2/(1 + np.exp(-10 * p))) -1
        #SOURCE ITER
        data_source = iter_source.next()
        img_source, lab_source = data_source
        batch_size = len(lab_source)
        lab_domain = torch.zeros(batch_size).long()
        model.zero_grad()
        if cuda:
            img_source = img_source.cuda()
            lab_source = lab_source.cuda()
            lab_domain = lab_domain.cuda()
        
        pred_label, domain_label = model(img_source, lambda_)

        lab_source = lab_source.type(torch.LongTensor)
        if cuda:
            pred_label = pred_label.cuda()
            domain_label = domain_label.cuda()
            lab_source = lab_source.cuda()
            
        error_source_label= loss_label(pred_label, lab_source)
        error_source_domain =loss_domain(domain_label, lab_domain)
        
        #TARGET ITER
        data_target = iter_target.next()
        img_target, lab_target = data_target
        batch_size = len(lab_target)
        lab_domain = torch.ones(batch_size).long()
        #model.zero_grad()
        if cuda:
            img_target = img_target.cuda()
            lab_target = lab_target.cuda()
            lab_domain = lab_domain.cuda()
            
        pred_label, domain_label = model(img_target, lambda_)
        
        error_target_domain = loss_domain(domain_label, lab_domain)
        
        error = error_source_domain + error_target_domain + error_source_label
        
        error.backward()
        optimizer.step()
            
    #Evaluate
    model.train(False)
    src_err.append(error_source_label.cpu().data.numpy())
    src_d_err.append(error_source_domain.cpu().data.numpy())
    tgt_d_err.append(error_target_domain.cpu().data.numpy())
    #torch.save(my_net, '{0}/mnist_mnistm_model1_epoch_{1}.pth'.format('/content/drive/MyDrive/DANN/models/', epoch))
    
    error_source_label,error_source_domain,error_target_label,error_target_domain,a,b,c,d = evaluate()
    print(f"Source label classifier: {error_source_label.item()}\n Source domain classifier: {error_source_domain.item()}\n Target label classifier: {error_target_label.item()}\n Target domain classifier:{error_target_domain.item()}\n Source classification acc:{100*a.item()}\n target_classification_acc:{100*b.item()}\n")# source_domain_acc:{100*c.item()}\n  target_domain_acc:{100*d.item()}\n")
    source_accu.append(a.cpu().data.numpy())
    target_accu.append(b.cpu().data.numpy())
    tgt_err.append(error_target_label.cpu().data.numpy())

    
        
        
        #800
        

In [155]:
n_epoch = len(src_err)
n_epoch

In [160]:
plt.plot(np.arange(0,n_epoch,1),np.array(src_err),label='Source')
plt.plot(np.arange(0,n_epoch,1),np.array(tgt_err),label='Target')
plt.xlabel("Epochs")
plt.ylabel("Error")
plt.title("Class Classification Error")
plt.legend()
#plt.savefig('/content/drive/MyDrive/DANN/err1.png')
plt.show()

plt.plot(np.arange(0,n_epoch,1),np.array(src_d_err),label='Source')
plt.plot(np.arange(0,n_epoch,1),np.array(tgt_d_err),label='Target')
plt.title("Domain Classification Error")
plt.xlabel("Epochs")
plt.ylabel("Error")
plt.legend()
#plt.savefig('err_alexnet.png')
plt.show()

plt.plot(np.arange(0,n_epoch,1),np.array(source_accu),label='Source')
plt.plot(np.arange(0,n_epoch,1),np.array(target_accu),label='Target')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Class Classification Accuracy")
plt.legend()
#plt.savefig('acc_alexnet.png')

In [156]:
np.array(target_accu).max()

In [157]:
mp = np.array(target_accu).argmax()

In [158]:
np.array(source_accu)[mp]

In [159]:
mp